In [87]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score,KFold
from xgboost import XGBRegressor
import seaborn
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y = train.SalePrice.reset_index(drop=True)
df_full = pd.concat([df, test]).reset_index(drop=True)
noncategorial = list(
df_full.select_dtypes(include=['float64', 'int64']).columns)
categorial = list(df_full.select_dtypes(include=['object']).columns)

In [88]:
df_full['Exterior1st'] = df_full['Exterior1st'].fillna(df_full['Exterior1st'].mode()[0])
df_full['LotFrontage'] = df_full.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
df_full['MSZoning'] = df_full.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
df_full["SalePrice"] = np.log1p(df_full['SalePrice'])

In [89]:
objects = []
for i in df_full.columns:
    if df_full[i].dtype == object:
        objects.append(i)

df_full.update(df_full[objects].fillna('None'))

df_full['LotFrontage'] = df_full.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Filling in the rest of the NA's

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in df_full.columns:
    if df_full[i].dtype in numeric_dtypes:
        numerics.append(i)
df_full.update(df_full[numerics].fillna(0))

In [90]:
#df_full['CondTot'] = (df_full['Condition1'] + df_full['Condition2']) / 2
df_full['OverTot'] = df_full['OverallQual'] + df_full['OverallCond']
df_full['TotalSF'] = df_full['TotalBsmtSF'] + df_full['1stFlrSF'] + df_full['2ndFlrSF']
df_full['haspool'] = df_full['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
df_full['has2ndfloor'] = df_full['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
df_full['hasgarage'] = df_full['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
df_full['hasbsmt'] = df_full['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
df_full['hasfireplace'] = df_full['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [91]:
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax,yeojohnson
numerics = [col for col in df_full.columns if df_full[col].dtype != object]
skew_f = df_full[numerics].apply(lambda x: skew(x)).sort_values(ascending=False)
highest_skew =  skew_f[skew_f > 0.5]
for i in highest_skew.index:
    df_full[i] = boxcox1p(df_full[i], boxcox_normmax(df_full[i] + 1))

In [92]:
df_full['YrBltAndRemod']=df_full['YearBuilt']+df_full['YearRemodAdd']
df_full['Total_sqr_footage'] = (df_full['BsmtFinSF1'] + df_full['BsmtFinSF2'] +
                                 df_full['1stFlrSF'] + df_full['2ndFlrSF'])
df_full['Total_Bathrooms'] = (df_full['FullBath'] + (0.5 * df_full['HalfBath']) +
                               df_full['BsmtFullBath'] + (0.5 * df_full['BsmtHalfBath']))
df_full['Total_porch_sf'] = (df_full['OpenPorchSF'] + df_full['3SsnPorch'] +
                              df_full['EnclosedPorch'] + df_full['ScreenPorch'] +
                              df_full['WoodDeckSF'])

In [93]:
# train = df_full.iloc[:1459]
# test = df_full.iloc[1460:]
# train['Sqr'] = train['SalePrice'] / train['LotArea']
# train['Sqr'] = train.groupby('Neighborhood')['Sqr'].transform(lambda x: x.median())
# d = {}
# for indice_fila, x_train in train.iterrows():
#     d.update({x_train['Neighborhood']:x_train['Sqr']})
# test['Sqr'] = 0.00
# for indice, x_test in test.iterrows():
#     test.loc[test.index == indice ,'Sqr'] = d[x_test['Neighborhood']]
# df_full['Sqr'] = pd.concat([train.Sqr,test.Sqr])

In [94]:
df_full['MSSubClass'] = df_full['MSSubClass'].apply(str)
df_full['MoSold'] = df_full['MoSold'].astype(str) 
df_full = pd.get_dummies(df_full).reset_index(drop=True)

In [95]:
outliers = [30, 88, 462, 631, 1322]
df_full = df_full.drop(df_full.index[outliers])
overfit = []
for i in df_full.columns:
    counts = df_full[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(df_full) * 100 > 99.94:
        overfit.append(i)

overfit = list(overfit)
#overfit.append('MSZoning_C (all)')

df_full = df_full.drop(overfit, axis=1).copy()
X = df_full.iloc[:len(y), :]
X_sub = df_full.iloc[len(X):, :]
#f = X.isna().sum()

In [96]:
from sklearn.linear_model import Ridge
clf = Ridge()
score = rmsle_cv(clf)
score


array([0.41371807, 0.4558529 , 0.45494386, 0.42854402, 0.46786576])

In [72]:
from sklearn.model_selection import KFold,cross_val_score
n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    rmse= np.sqrt(-cross_val_score(model, X.values, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [23]:
final_df_full.to_csv('prepared.csv',index=False)